In [ ]:
%load_ext autoreload
%autoreload 2

# # Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / "eye-ai-ml"))
sys.path.insert(0, str(Path.home() / "eye-ai-exec"))

In [ ]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration, DatasetVersion
from deriva_ml import MLVocab as vc
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

In [ ]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

In [ ]:
datasets = [
    '4-YQVM', # Healthy USC data
    ]

to_be_download = []
for dataset in datasets:
    ds_dict = {
        'rid': dataset,
        'materialize':True,
        'version':EA.dataset_version(dataset_rid=dataset),
    }
    to_be_download.append(ds_dict)

workflow_instance = EA.create_workflow(
    name="Cropping Image",
    workflow_type="Image Cropping"
)

config = ExecutionConfiguration(
    datasets=to_be_download,
    assets = ['2-4JR6',],
    workflow=workflow_instance,
    description="Instance of cropping multimodal images.")

execution = EA.create_execution(config)

In [ ]:
print(execution)

In [ ]:
execution._working_dir

In [ ]:
ds_bag = execution.datasets[0]
crop_image_model =  execution.asset_paths["Execution_Asset"][0]

In [ ]:
df = ds_bag.get_table_as_dataframe("Image")

In [ ]:
df.rename(columns={'RID': 'RID_Image'}, inplace=True)

In [ ]:
from pathlib import Path
working_dir = Path("~/working_dir")
csv_path = execution._working_dir / "df.csv"
df.to_csv(csv_path, index=False)

In [ ]:
table_name = 'Image'
EA.find_features(table_name)

In [ ]:
ImageAnnotationFeature = EA.feature_record_class("Image", "Annotation")

In [ ]:
output = execution._working_dir / "image_svg_output"
output.mkdir(parents= True, exist_ok=True)

In [ ]:

from models.vgg19.vgg19_disk_crop_predict import preprocess_and_crop
with execution.execute() as exec:
    preprocess_and_crop(
      ds_bag,
      csv_path,
      execution._working_dir / 'output_train.csv',
      'template.jpg',
      str(output),
      crop_image_model,
      "2-NK8E",
      "Optic Nerve",
      False
      )
    

In [ ]:
angle_2_only = EA.filter_angle_2(ds_bag)

In [ ]:
csv_output  = Path(execution._working_dir / 'output_train.csv')

In [ ]:
df_output = pd.read_csv(csv_output)

In [ ]:
filtered_resulted_df = df_output[df_output["Image RID"].isin(angle_2_only["RID"])]

In [ ]:
cropping_func_map_output = dict(zip(filtered_resulted_df["Image RID"], filtered_resulted_df["Worked Image Cropping Function"]))

In [ ]:
import shutil 

saved_svg_names = filtered_resulted_df["Saved SVG Name"].values

bounding_box_file_paths = []
for file_name in os.listdir(output):
    if file_name in saved_svg_names:
        full_path_name = os.path.join(output, file_name)
        
        bounding_box_file = execution.asset_file_path(
            "Fundus_Bounding_Box", full_path_name, 
        )
        bounding_box_file_paths.append(bounding_box_file)

In [ ]:
image_rids = []
angle_2_rids = filtered_resulted_df["Image RID"].values

for file_name in os.listdir(output):
    rid = file_name.split("_")[1].split('.')[0]
    if rid in angle_2_rids:
        image_rids.append(rid)
len(image_rids)

In [ ]:
image_bounding_box_feature_list = []

for image_rid, asset_file_path in zip(image_rids, bounding_box_file_paths):
    expected_name = f"Cropped_{image_rid}.svg"
    actual_name = os.path.basename(asset_file_path)
    assert expected_name == actual_name, f"Expected {expected_name}, got {actual_name}"

    annotation_func = cropping_func_map_output.get(image_rid)
    if annotation_func:
        image_bounding_box_feature_list.append(
            ImageAnnotationFeature(
                Image=image_rid,
                Execution=execution.execution_rid,
                Fundus_Bounding_Box=asset_file_path,
                Annotation_Function=annotation_func,
                Annotation_Type='Optic Nerve',
            )
        )


In [ ]:
with execution.execute() as execution:
    execution.add_features(image_bounding_box_feature_list)

In [ ]:
execution.upload_execution_outputs()